In [102]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup

In [125]:
#read in csv to dataframe
df = pd.read_csv('spreadspoke_scores.csv')
#Eliminate observations that do not have the data we need
df= df[(df.spread_favorite.notnull()) & (df.over_under_line.notnull())& (df.score_home.notnull()) &(df.score_away.notnull())&(df.weather_temperature.notnull())&(df.weather_wind_mph.notnull())]
del df['weather_humidity'], df['stadium_neutral'], df['stadium'], df ['weather_detail']
#Only want post 1987 data to avoid last NFL strike
df= df[df.schedule_season>1987]
#Fill in weather detail data
#df.weather_detail=df.weather_detail.fillna('CLEAR')- considered this but weather detail column was too unpopulated


CLEAR                   5659
DOME                    1834
Rain                     106
DOME (Open Roof)          56
Fog                       28
Rain | Fog                22
Snow                      14
Snow | Fog                 5
Snow | Freezing Rain       1
Name: weather_detail, dtype: int64

In [119]:
## teamSwitch takes a string of the name of any football team from the past 30 years and returns the current abbreviation
def teamSwitch (full):
    abv=''
    if full == 'New York Jets' or full== 'New York Giants':
        for let in range(len(full)):
            if let == 0:
                abv+=full[let]
            elif full[let]==' ':
                abv+=full[let+1]
    elif full.startswith('New ') or full.startswith('Kans') or full[6:9]=='Bay' or full.endswith('49ers'):
        for let in range(len(full)):
            if let == 0:
                abv+=full[let]
            elif full[let]==' ':
                abv+=full[let+1]
                return abv
    elif full.endswith('Oilers'):
        return "TEN"
    elif full.endswith('Cardinals'):
        return 'ARI'
    elif full.endswith('Rams'):
        return 'LAR'
    elif full.endswith('Chargers'):
        return 'LAC'
    elif full.endswith('Raiders'):
        return 'OAK'
    elif full[0:4]== 'Jack':
        return 'JAX'
    else:
        return full[0:3].upper()
    return abv
ppdf= df.copy()
#apply teamSwitch to data
ppdf.team_home=ppdf.team_home.apply(teamSwitch)
ppdf.team_away=ppdf.team_away.apply(teamSwitch)
#create a column to show if the home team is favored
homefave= ppdf.team_home==ppdf.team_favorite_id
ppdf['home_fave']= homefave
#create a column to show the total points scored by both teams
ppdf['total']= ppdf.score_home+ppdf.score_away

In [120]:
#Create a dict, result, to track how teams do compared to the spread. If favorite wins, the number is negative and the number ispositive if underdog wins
#Initialize empty dict
result= {}
#loop through dataFrame index to populate dict with keys from the index  and values from columns
for x in ppdf.index:
    if ppdf.loc[x]['home_fave']== True:
        result[x]=ppdf.loc[x]['score_away']-ppdf.loc[x]['score_home']
    else:
        result[x]=ppdf.loc[x]['score_home']-ppdf.loc[x]['score_away']


In [127]:
#create new column for result
result = pd.Series(result)
ppdf['results']=result
#using the new results column, create a bolean series to become a new column to indicate if the spread ws covered
cover =ppdf.results<ppdf.spread_favorite
ppdf['favorite_cover']=cover
#create a coolean series to make a column to indicate if the over was reached
ppdf['over_under_line']=ppdf['over_under_line'].astype(float)
over= ppdf['total']>ppdf['over_under_line']
ppdf['over_cover']= over
#rearrange columns for more readable DataFrame
ppdf=ppdf[['schedule_date', 'schedule_season', 'schedule_week', 'team_home','score_home', 'score_away','team_away','team_favorite_id', 'spread_favorite', 'results', 'favorite_cover', 'over_under_line', 'total', 'over_cover','weather_temperature','weather_wind_mph','schedule_playoff']]
ppdf.describe()

,schedule_season,score_home,score_away,spread_favorite,results,over_under_line,total,weather_temperature,weather_wind_mph
count,7725.000000,7725.000000,7725.000000,7725.000000,7725.000000,7725.000000,7725.000000,7725.000000,7725.000000
mean,2003.473398,22.655663,19.939806,-5.437605,-5.391845,41.963327,42.595469,60.136052,7.015663
std,8.864199,10.328487,10.017573,3.453854,13.809755,4.788337,14.200371,15.520104,5.667636
min,1988.000000,0.000000,0.000000,-26.500000,-59.000000,28.000000,3.000000,-6.000000,0.000000
25%,1996.000000,16.000000,13.000000,-7.000000,-14.000000,38.000000,33.000000,50.000000,1.000000
50%,2004.000000,22.000000,20.000000,-4.500000,-4.000000,42.000000,42.000000,64.000000,7.000000
75%,2011.000000,30.000000,27.000000,-3.000000,3.000000,45.000000,51.000000,72.000000,11.000000
max,2018.000000,62.000000,59.000000,0.000000,45.000000,63.500000,106.000000,97.000000,40.000000
